# Transitions Notebook 

In [6]:
import cProfile
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
# %matplotlib qt

import seaborn as sns
import bk.load
import bk.compute
import bk.plot

import os

import matplotlib as mpl
mpl.rcParams["svg.fonttype"] = "none"

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_610588/2123766811.py", line 6, in <cell line: 6>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/IP

# Utils Functions

In [7]:
def plot_loghist(x, bins):
    hist, bins = np.histogram(x, bins=bins)
    logbins = np.logspace(np.log10(bins[0]), np.log10(bins[-1]), len(bins))
    plt.hist(x, bins=logbins)
    plt.xscale("log")


In [8]:
def transition(states, template, epsilon=0):
    """
    author: BK
    states : dict of nts.Interval_set
    template : list of state.
    epsilon : int, will drop any 
     in which there is an epoch shorter than epsilon 's'
    This function will find transition that match the template 
    """
    if epsilon is list:
        print("eplist")
    long = pd.DataFrame()
    for s, i in states.items():
        i["state"] = s
        long = pd.concat((i, long))
        del i["state"]
    order = np.argsort(long.start)
    long = long.iloc[order]

    transition_times = []
    transition_intervals = []
    for i, s in enumerate(long.state):
        tmp = list(long.state[i: i + len(template)])
        if tmp == template:
            tmp_transition = long.iloc[i: i + len(template)]
            #             print(d.iloc[i:i+len(template)])
            length = (tmp_transition.end - tmp_transition.start) / 1_000_000
            if np.any(length.values < epsilon):
                continue
            tmp_pre = np.array(tmp_transition.end[:-1])
            tmp_post = np.array(tmp_transition.start[1:])
            tmp_times = np.mean([tmp_pre, tmp_post], 0)

            transition_intervals.append(
                [tmp_transition.start.iloc[0], tmp_transition.end.iloc[-1]]
            )
            transition_times.append(tmp_times)

    transition_times = np.array(transition_times)
    transition_intervals = np.array(transition_intervals)
    transition_intervals = nts.IntervalSet(
        start=transition_intervals[:, 0],
        end=transition_intervals[:, 1],
        force_no_fix=True,
    )
    return transition_intervals, transition_times

In [12]:
def main(base_folder = '/mnt/icybox', local_path="Rat08/Rat08-20130713", *args):
    # Variables :
    plot = False
    save = False

    print(args)
    if args == None:
        transition_interest = [
            ["Rem", "sws", "Rem"]
        ]  # Template of transitions to follow.
        bin_epochs = [12, 30, 12]
        n_event = 3
    else:
        transition_interest = [args[0]]
        bin_epochs = args[1]
        n_event = args[2]

    bk.load.current_session_linux(
        base_folder=base_folder, local_path=local_path)
    neurons, metadata = bk.load.loadSpikeData(bk.load.path)
    states = bk.load.states()

    for i, transition_name in enumerate(
        transition_interest
    ):  # For each kind of transitions
        print(f"Computing {transition_name} transitions")
        intervals, timing = transition(
            states, transition_name, epsilon=50
        )  # Find the timing of the transitions
        print(f"Found {len(intervals)} transitions")

        transition_activity = compute_transition_activity(
            neurons, intervals, timing, bin_epochs, n_event
        )

        if plot:
            print(
                np.mean(transition_activity[metadata.Region == "BLA"], 2).T.shape)
            y_bla = scipy.stats.zscore(
                np.mean(transition_activity[metadata.Region == "BLA"], 2).T, 0
            )
            y_hpc = scipy.stats.zscore(
                np.mean(transition_activity[metadata.Region == "Hpc"], 2).T, 0
            )

            plt.figure()
            plt.plot(y_bla, "green")
            plt.plot(y_hpc, "blue")

            plt.title(f"Rat:{bk.load.rat}, Day{bk.load.day} {bk.load.session}")
        print(os.getcwd())
        # sanity_check(neurons,states,intervals,timing)

        return np.array(transition_activity), metadata, bin_epochs


In [13]:
def merge(batch, name, discard):
    transitions_activity = []
    metadata = []
    n_transitions = 0
    n_sessions = 0
    for i, b in batch.items():
        if i not in discard:
            #          if True:
            transitions_activity.append(np.mean(b[0], 2))
            metadata.append(b[1])
            n_transitions += b[0].shape[2]
            n_sessions += 1
            bins_epochs = b[2]

    s = np.append(0, np.cumsum(bins_epochs))[:-1]
    e = np.cumsum(bins_epochs)-1

    intervals = np.array([s, e]).T
    merged = {'name': name,
              'average_transition': np.vstack(transitions_activity),
              'metadata': pd.concat(metadata),
              'n_transitions': n_transitions,
              'n_sessions': n_sessions,
              'intervals': intervals}

    merged['metadata'].Type[merged['metadata'].Region == 'CeCM'] = 'Int'
    return merged


In [14]:
discard = [
    "Rat08-20130718",
    "Rat09-20140408",
    "Rat09-20140409",
    "Rat10-20140619",
    "Rat10-20140620",
    "Rat10-20140622",
]


In [15]:
bk.load.current_session_linux()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_610588/3346676129.py", line 1, in <cell line: 1>
    bk.load.current_session_linux()
NameError: name 'bk' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/billel/.local/lib/python3.8/site-packages/